${\bf Simon's \, Algorithm}$<br><br>
Imagine that a large number of people are lined up in a row. Also, that they’re spaced out really far from each other. They're so far apart that you can only clearly see one person at a time. 

Also imagine that each person has an identical twin who’s located somewhere else in the line. And, the number of people between each pair of twins is the same. For example, there might be 10 other people in between every pair of twins.

How could you figure out what the hidden amount of spacing is? You could start at the first person and then look down the line, one person at a time, until you found someone who looked exactly like the first person. But, that would be time consuming. It'd be nice if there was a faster way to find the spacing. 

This problem is actually related to (but is <i>not</i> identical to) an important early quantum algorithm: Simon's algorithm. It was created by Daniel Simon in 1994. It was the first ever quantum alogrithm that performed <i>exponentially better</i> than the best classical algorithm for the same problem. It also inspired Peter Shor to come up with his famous factoring algorithm (i.e., Shor's algorithm). The publication of Shor's algorithm led to an explosion in the growth of quantum computing. So, Simon's algorithm holds a special place in the history of quantum computing.  

This notebook explains Simon's algorithm. It adds to the many excellent explanations that are already out there. For example, 
<a href="https://learn.qiskit.org/course/ch-algorithms/simons-algorithm">IBM Qiskit's explanation</a>, <a href="https://aws.amazon.com/blogs/quantum-computing/simons-algorithm/">AWS's explanation</a> (with Daniel Simon himself), and <a href="https://cs.uwaterloo.ca/~watrous/QC-notes/QC-notes.06.pdf">this explanation</a> from Professor John Watrous of the University of Waterloo.

The explanation below assumes a prior knowledge of the following concepts:
- statevectors
- measurement postulate in quantum physics
- Hadamard gate
- concept of an oracle

${\bf What}$  ${\bf problem}$ ${\bf does}$   ${\bf Simon's}$  ${\bf algorithm}$  ${\bf solve?}$ 

Consider a function $f(x)$ that maps an $n$-bit string to another $n$-bit string, $f \colon \{ 0,1 \}^{n} \rightarrow \{ 0,1 \}^{n} $ 

The function $f$ is 2:1. That is, for each output, there are two different inputs that yield this output. Furthermore, these two inputs, $a$ and $b$, are related via the following mathematical rule:
$b = a \oplus s$, where $s$ is a $n$-bit string.
Note that $\oplus$ means bitwise addition modulo 2. That is, for the bit strings $c = c_{0} c_{1} ... c_{n}$ and $d=d_{0} d_{1} ... d_{n}$, $c \oplus d = \bigl( (c_{0} + d_{0}) {\rm mod} \, 2\bigr) \, \bigl( (c_{1} + d_{1}) {\rm mod} \, 2\bigr)... \bigl( (c_{n} + d_{n}) {\rm mod} \, 2\bigr)$  

For example, for $n=2$ and $s=01$, we have

$00 \oplus 01 = 01$ and so $f(00) = f(01)$

and $10 \oplus 11 = 01$ and so $f(10) = f(11)$.

Assume what we don't know the value of $s$. Also, that we have in front of us a black box that we can input any $n$-bit string, $x$, into and have it output the value of $f(x)$. What's the most efficient way to determine $s$? And, how many times do we need to query the black box?

Classically, about the best we can do is to calculate $f(x)$ for some value of $x$ and then input other bit strings, one by one, until the black box outputs the same value as it did initially. In the worst case scenario, we need to try just over half of the inputs, $2^{n-1}+1$, until $f(x)$ returns the same value. We can actually do a bit better than this but, still, the best possible classical algorithm scales exponentially with the number of input bits, $\Omega(2^{n/2})$. 

${\bf Quantum}$  ${\bf Solution}$ ${\bf to}$   ${\bf Simon's}$  ${\bf problem}$ <br>
Simon's algorithm involve two computing registers, an input register and an output one. We'll show the quantum state of the input register first and the state of the output register second. For example, 

$|\psi \rangle = |\psi_{1} \rangle |\psi_{2} \rangle$

means that the input register in the state $|\psi_{1} \rangle$ and the output register in the state $|\psi_{2} \rangle$.

<b>STEP 1</b> <br> 
Apply a Hadamard gate to each qubit in the input register. <br>
$|\psi \rangle = \Bigl( H^{\otimes n} |0 \rangle^{\otimes n} \Bigr) |0 \rangle^{\otimes n}$

$|\psi \rangle = \frac{1}{\sqrt{2^{n}}} \Bigl( ( |0 \rangle + |1 \rangle)  ( |0 \rangle + |1 \rangle) ... ( |0 \rangle + |1 \rangle)\Bigr)|0 \rangle^{\otimes n}$

$= \frac{1}{\sqrt{2^{n}}}\Bigl( |000...0 \rangle  + |000...1 \rangle + |000...10 \rangle + ... | 111....1 \rangle \Bigr) |0 \rangle^{\otimes n}$

It's cumbersome to write down (and read) the values of all the qubits. So, we'll switch to a more compact representation. 

${\bf Converting}$  ${\bf from}$ ${\bf Binary}$   ${\bf Representation}$  ${\bf to}$  ${\bf Decimal}$ ${\bf Representation}$

Each string of $n$ qubit values above is a binary number with $n$ binary digits. If we switch from a binary representation to a decimal one, the representaton of $|\psi \rangle$ becomes more compact and easier to grasp. 

This yields

$|\psi \rangle = \frac{1}{\sqrt{2^{n}}} \Sigma_{x=0}^{2^{n}-1}  |x \rangle |0 \rangle$

The variable $x$ is a decimal number that represents a $n$-bit binary number. For example, when $n$=3, $x=2$ represents the bit string 010 ($\equiv 2$).


<b>STEP 2</b> <br> 
Apply the oracle using the input register as the input and writing the output to the output register.

$|\psi \rangle = \frac{1}{\sqrt{2^{n}}} \Sigma_{x=0}^{2^{n}-1} |x \rangle ) |f(x) \rangle$ 

<b>STEP 3</b> <br> 
Measure the output register. You'll get a particular value $f(x)$. But, it doesn't matter which one you get.<br><br>
Because $f$ is a 2:1 function, two different inputs will both yield the value of $f(x)$ that we measure. Let's call these inputs $a$ and $b$. The input register then becomes <br>
$|a \rangle + | b \rangle$

<b>STEP 4</b> <br> Apply a Hadamard gate to each qubit in the first register. This gives

$H^{\otimes n} \left( |a \rangle + | b \rangle \right)$

Converting $|a\rangle$ and $|b\rangle$ back to a binary representation yields

$H^{\otimes n} \left( | a_{0} \rangle | a_{1} \rangle | a_{2} \rangle ... | a_{n-1} \rangle \right) + H^{\otimes n} \left( | b_{0} \rangle | b_{1} \rangle | b_{2} \rangle ... | b_{n-1} \rangle \right)$

where $a_{i} (b_{i})$, for $i$ = 0 ... $n$-1 is the $i^{\rm th}$ binary digit in the binary representation of $a$ ($b$) 

Now, $H | 0 \rangle = \frac{1}{\sqrt{2}} (|0\rangle + | 1 \rangle)$

and

$H | 1 \rangle = \frac{1}{\sqrt{2}} (|0\rangle - | 1 \rangle)$

So, we only get a minus sign in front of a qubit statevector when:
1. the statevector prior to applying the Hadamard gate is $| 1 \rangle$ and
2. the final qubit statevector is $| 1 \rangle$

To represent this behaviour compactly, consider the binary operation called the bitwise dot product:<br>
$e \cdot f = \bigl( e_{0} \times f_{0} + e_{1} \times f_{1} ... + e_{n-1} \times f_{n-1} \bigr) ({\rm mod}\, 2)$, where $e$ and $f$ are both bit strings.

The above behaviour with the Hadamard gate and minus signs mirrors the behaviour of the bitwise dot product as<br>
$\bigl($ qubit value (initial) $\cdot$ qubit value (final) $\bigr)$ <br> 
yields <br>
0 $\cdot$ 0 = 0 <br>
0 $\cdot$ 1 = 0 <br>
1 $\cdot$ 0 = 0 <br>
1 $\cdot$ 1 = 1 
   
So, the state of the input register becomes

$\frac{1}{\sqrt{2^{n}}} \Sigma_{x=0}^{2^{n}-1} \left( (-1)^{a \cdot x} +  (-1)^{b \cdot x}  \right)  |x \rangle$ 

Note that the coefficient $\bigl( (-1)^{a \cdot x} +  (-1)^{b \cdot x} \bigr)$ is only non-zero when $a . x  = b.z$. That is, when there's constructive interference involving the two coefficients.

<b>STEP 5</b> <br>
Measure all the qubits in the input register.
Let the measured value that we get be $z_{meas}$. From the property of the coefficients indentified at the end of Step 4, we know that $a.z_{meas} = b.z_{meas}$

But, as $b = a \oplus s$

$a.z_{meas} = (a \oplus s).z_{meas}$

Expanding the brackets on the right-hand side gives <br>
$a.z_{meas} = a.z_{meas} \oplus s.z_{meas}$

Subtracting $a.z_{meas}$ from both sides gives <br> 
$s \cdot z_{meas} = 0 \; ({\rm mod} \; 2)$. 

If we repeat all five steps approximately $n$ times, there's a reasonable chance that we'll measure different values for $z_{\rm meas}$ each time. This will give us a system of $n$ simultaneous linear equations that we can then solve. 
This is Simon's algorithm. It solves the problem in $O(n)$ computations.
Recall that the best classical algorithm takes $\Omega \bigl( {\rm exp}(n/2) \bigr)$ computations.

The difference between these two functions is the expontential gap mentioned at the start.

Simon's algorithm is exponentially more efficient than you can do classically.

Note that, technically, in the statement of the problem that Simon's algorithm solves, $f$ is either a 2:1 function ${\it or}$ a 1:1 function. This doesn't make much difference though as we can consider the 1:1 case as one where $s=0$.

${\bf Qiskit \ code \ for \ Running \ Simon's \ Algorithm}$

${\bf Note:}$ This code creates and runs a quantum circuit that produces ${\it one}$ equation of the form $s \cdot z_{meas} = 0$. To run the full algorithm, you need to run it $O(n)$ times to generate $n$ the set of $n$ simultaneous equations mentioned earlier.

In [8]:
'''
This code implements Simon's algorithm in Qiskit for two qubits.
'''

from qiskit import QuantumCircuit, execute, Aer, QuantumRegister,ClassicalRegister

#define the hidden "period"
s = '01'
print("s=",s,"(unknown string)\n")

n = len(s)
simons_circuit = QuantumCircuit(2*n, 2*n)

#STEP 1: Apply Hadamard gates to input register
simons_circuit.h(range(n))    

#STEP 2: apply oracle to circuit
#this oracle implement the 2:1 function f(x): f(00) = f(01) = 00 and f(10) = f(11) = 10
simons_circuit.cnot(0,2)

#STEP 3: Measure both qubits in the output register & store results in the classical register
simons_circuit.measure(range(n,2*n), range(n,2*n))

#STEP 4: Apply second set of Hadamard gates to input register
simons_circuit.h(range(n))

#STEP 5
#Measure input qubits and store the results in classical register
simons_circuit.measure(range(n), range(n))

#define a backend & run quantum circuit on it
backend = Aer.get_backend('qasm_simulator')
job = execute(simons_circuit, backend, shots=1)
result = job.result()
counts = result.get_counts(simons_circuit)

print("counts=",counts,)

measured_input_string = counts.keys()

print("\nString measured in input registers:")
print("z_meas = ",list(measured_input_string)[0][3],list(measured_input_string)[0][2])
print("(Note that the first bit in z_meas is the value measured for qubit 0 (i.e., the first qubit).\n The second bit in z_meas is the value measured for qubit 1.)")

s= 01 (unknown string)

counts= {'0101': 1}

String measured in input registers:
z_meas =  1 0
(Note that the first bit in z_meas is the value measured for qubit 0 (i.e., the first qubit).
 The second bit in z_meas is the value measured for qubit 1.)
